In [3]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, holdout_split
from sklearn.metrics import mean_squared_log_error

In [4]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)
traintestval, holdout = holdout_split(dfraw)
trainval, test = holdout_split(traintestval)

In [5]:
test

,state,year,name,M/F,count
0,US,2000,Madison,F,19968
1,US,2000,Olivia,F,12854
2,US,2000,Sydney,F,10244
3,US,2000,Jennifer,F,9389
4,US,2000,Amanda,F,8557
...,...,...,...,...,...
212633,US,2019,Zaheen,M,5
212634,US,2019,Zahi,M,5
212635,US,2019,Zaymar,M,5
212636,US,2019,Zeo,M,5


In [6]:
def get_all_known_names(data):

    names = data.groupby(['state', 'name', 'M/F']).size().reset_index()
    names = names[['state', 'name', 'M/F']]
    return names

In [7]:
def evaluate(predictor, data_held_out, first_year_to_predict, metric='msle'):

    most_recent_year = data_held_out['year'].max()
    
    years_to_predict = range(first_year_to_predict, most_recent_year+1)

    # only allow the model to see data from before the year to predict
    historical_data = data_held_out[data_held_out['year'] < first_year_to_predict]

    # get our model's predictions
    predictions = predictor.predict(historical_data, years_to_predict)

    all_known_names = get_all_known_names(historical_data)

    for year_to_predict in years_to_predict:

        print(f'Predictions for {year_to_predict}:')

        names_to_predict = all_known_names.copy()
        names_to_predict['year'] = year_to_predict
        # display(names_to_predict)

        observed = names_to_predict.merge(data_held_out, how='left', on=['state', 'name', 'M/F', 'year'])

        # for now, fill in missing values with 2, same as FiveThirtyEight did;
        # reasoning: missing values could be 0 to 4, so average is 2
        observed['y'] = observed['count'].fillna(2)
        # observed = observed.rename(columns={'count': 'count_true'})
        # display(observed)

        score_df = observed.merge(predictions, how='left', on=['state', 'name', 'M/F', 'year'], suffixes=('_true', '_pred'))
        # display(score_df)

        y_true = score_df['y_true']
        y_pred = score_df['y_pred']

        if metric == 'msle':
            loss = mean_squared_log_error(y_true, y_pred)
            print(f'Loss: {loss}')

        if metric == 'rank':
            y_true = y_true.rank()
            y_pred = y_pred.rank()
            score = np.sum(np.abs(y_true-y_pred))/(len(y_true)*(len(y_true)-1))
            print(f'Score: {score}')

In [8]:
class DummyPredictor():

    def __init__(self, strategy='naive'):
        self.strategy = strategy
    
    def predict(self, historical_data, years_to_predict):

        all_known_names = get_all_known_names(historical_data)

        predictions = []
        previous_year_data = historical_data[historical_data['year'] == years_to_predict[0] - 1].drop(columns=['year'])

        for year_to_predict in years_to_predict:

            prediction = all_known_names.copy()
            prediction['year'] = year_to_predict

            if self.strategy == 'naive':
                prediction = prediction.merge(previous_year_data, how='left', on=['state', 'name', 'M/F'])
                prediction['y'] = prediction['count'].fillna(2)
                # display(prediction)
            elif self.strategy == 'mean':
                prediction['y'] = previous_year_data['count'].mean()
                # display(prediction)

            predictions.append(prediction)

        predictions = pd.concat(predictions, ignore_index=True)
        # display(predictions)

        return predictions

In [43]:
first_year_to_predict = 2003
evaluate(predictor=DummyPredictor(strategy='naive'), data_held_out=test, first_year_to_predict=first_year_to_predict, metric='rank')

Predictions for 2003:
Score: 0.08482118448949341
Predictions for 2004:
Score: 0.09198609495673483
Predictions for 2005:
Score: 0.09647119439296147
Predictions for 2006:
Score: 0.10225763022659949
Predictions for 2007:
Score: 0.10981467396895285
Predictions for 2008:
Score: 0.11692768051699107
Predictions for 2009:
Score: 0.12410667680667863
Predictions for 2010:
Score: 0.1318546134840307
Predictions for 2011:
Score: 0.13774315484104016
Predictions for 2012:
Score: 0.14591413349394286
Predictions for 2013:
Score: 0.15450041025792344
Predictions for 2014:
Score: 0.15751621154662263
Predictions for 2015:
Score: 0.16642579465234086
Predictions for 2016:
Score: 0.17094297104442235
Predictions for 2017:
Score: 0.17676659077902007
Predictions for 2018:
Score: 0.18370178885272045
Predictions for 2019:
Score: 0.18764221423521227
Predictions for 2020:
Score: 0.1922316889481903
Predictions for 2021:
Score: 0.19663804796936848
Predictions for 2022:
Score: 0.20246006552510978


In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor

class MyPredictor():

    def __init__(self):
        
        # params
        cols_to_keep = ['M/F', 'sum', 'median_age', 'thisyear_count']
        categorical_features = ['M/F']
        max_leaf_nodes = 16 # 16
        max_iter = 100 # 100
        loss = 'absolute_error' # abs better than default

        categorical_features = [True if f in categorical_features else False for f in cols_to_keep]
        # print(categorical_features)
        
        self.pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    # ('category_encoder', LabelEncoder(), categorical_features),
                    ('cols_to_keep', 'passthrough', cols_to_keep),
                ], remainder='drop'),
            HistGradientBoostingRegressor(
                random_state=0,
                categorical_features=categorical_features,
                max_leaf_nodes=max_leaf_nodes,
                max_iter=max_iter,
                loss=loss
            )
        )

        self.gender_encoding = {'M': 0, 'F': 1}

    def preprocess(self, df, latest_known_year):

        # find median age of people with name, 
        # total born with that name,
        # and latest year's count

        df = df.copy()
        df = df.sort_values(by='year')
        df['cumsum'] = df.groupby(['state', 'name', 'M/F'])['count'].cumsum()
        df['sum'] = df.groupby(['state', 'name', 'M/F'])['count'].transform('sum')
        # display(df[(df['name'] == 'Millie') & (df['M/F'] == 'M')])

        medians = df[df['cumsum'] >= df['sum']/2]
        medians = medians.drop_duplicates(subset=['state', 'name', 'M/F'], keep='first')
        medians['median_age'] = latest_known_year - medians['year']
        # display(medians[medians['name'] == 'Madison'])

        thisyear = df[df['year'] == latest_known_year][['state', 'name', 'M/F', 'count']].rename(columns={'count': 'thisyear_count'})
        df2 = medians.merge(thisyear, how='left', on=['state', 'name', 'M/F']).rename(columns={'year': 'median_year'})
        df2['thisyear_count'] = df2['thisyear_count'].fillna(0) # might want to shift this to 2 and fill in 2s for missing years? or maybe not
        # display(df2)
        # display(df2.groupby(['state','name','M/F']).ngroups)

        # change M/F to 0/1 so it works with various models
        # (even HistGradientBoostingRegressor, which accepts categorical values,
        # still needs those values to be numbers not strings)
        df2['M/F'] = df2['M/F'].map(self.gender_encoding)

        return df2
    
    def fit(self, historical_data, first_year_to_predict, years_to_fit=1, weight_decay=0.9):
        # first things first, we don't want to know about future data
        historical_data = historical_data[historical_data['year'] < first_year_to_predict]
        # at this point the data we don't want to know should be inaccessible

        X_all = pd.DataFrame()
        y_all = pd.Series()

        # each year_to_fit is the year that's essentially our y for that loop
        for year_to_fit in range(first_year_to_predict - years_to_fit, first_year_to_predict):

            # now we "know" even less for X
            X = historical_data[historical_data['year'] < year_to_fit]
            y = historical_data[historical_data['year'] == year_to_fit]

            X = self.preprocess(X, year_to_fit - 1)
            y = y[['state', 'name', 'M/F', 'count']].rename(columns={'count': 'y'})
            y['M/F'] = y['M/F'].map(self.gender_encoding)

            data = X.merge(y, how='left', on=['state', 'name', 'M/F'])
            data['y'] = data['y'].fillna(0)
            # display(data)

            X = data.drop(columns=['y'])
            y = data['y']
            X['sample_weight'] = weight_decay ** (first_year_to_predict - year_to_fit)

            X_all = pd.concat([X_all, X], ignore_index=True)
            y_all = pd.concat([y_all, y], ignore_index=True)
        
        temp = X_all.copy()
        temp['y'] = y_all
        display(temp)

        sample_weights = X_all['sample_weight']
        X_all = X_all.drop(columns=['sample_weight'])

        self.pipe.fit(X_all, y_all, **{'histgradientboostingregressor__sample_weight': sample_weights})
        # this seems like a silly way to pass params to individual steps of the pipeline, but it's true. See: https://stackoverflow.com/questions/36205850/sklearn-pipeline-applying-sample-weights-after-applying-a-polynomial-feature-t

    def predict(self, historical_data, years_to_predict):

        # all_known_names = get_all_known_names(historical_data)

        predictions = []

        for year_to_predict in years_to_predict:

            display(historical_data)

            df = self.preprocess(historical_data, year_to_predict - 1)
            # df = self.preprocess(historical_data, years_to_predict[0] - 1)

            df['y'] = self.pipe.predict(df)

            df['year'] = year_to_predict
            # display(df)

            # if we want to simply, do the following; 
            # but for now, might be useful to see all data displayed.
            # df = df[['state', 'year', 'name', 'M/F', 'y']]

            predictions.append(df)

            assumed_new_year_of_historical_data = df[['state', 'year', 'name', 'M/F', 'y']].rename(columns={'y': 'count'})
            assumed_new_year_of_historical_data['M/F'] = assumed_new_year_of_historical_data['M/F'].map({v: k for k, v in self.gender_encoding.items()})
            historical_data = pd.concat([historical_data, assumed_new_year_of_historical_data], ignore_index=True)

        predictions = pd.concat(predictions, ignore_index=True)

        # we have to reverse the mapping to send our predictions
        # (at least the way we currently have it set up)
        predictions['M/F'] = predictions['M/F'].map({v: k for k, v in self.gender_encoding.items()})

        predictions.loc[predictions['y'] < 4.5, 'y'] = 2

        display(predictions)
        display(predictions[predictions['y'] < 4.5])

        return predictions

In [46]:
first_year_to_predict = 2003
my_predictor = MyPredictor()
my_predictor.fit(historical_data=trainval, first_year_to_predict=first_year_to_predict, years_to_fit=30)
evaluate(predictor=my_predictor, data_held_out=test, first_year_to_predict=first_year_to_predict, metric='rank')

,state,median_year,name,M/F,count,cumsum,sum,median_age,thisyear_count,sample_weight,y
0,US,1937,Major,1,6,6,11,35,0.0,0.042391,0.0
1,US,1937,Loella,1,5,5,10,35,0.0,0.042391,0.0
2,US,1937,Nazaria,1,5,5,5,35,0.0,0.042391,0.0
3,US,1937,Coleman,1,5,5,5,35,0.0,0.042391,0.0
4,US,1937,Illah,1,5,5,5,35,0.0,0.042391,0.0
...,...,...,...,...,...,...,...,...,...,...,...
418854,US,2001,Aziyah,1,11,11,11,0,11.0,0.900000,19.0
418855,US,2001,Symia,1,12,23,23,0,12.0,0.900000,12.0
418856,US,2001,Syniah,1,12,17,17,0,12.0,0.900000,20.0
418857,US,2001,Viktoriya,1,12,17,17,0,12.0,0.900000,0.0


,state,year,name,M/F,count
0,US,2000,Madison,F,19968
1,US,2000,Olivia,F,12854
2,US,2000,Sydney,F,10244
3,US,2000,Jennifer,F,9389
4,US,2000,Amanda,F,8557
...,...,...,...,...,...
202313,US,1937,Son,M,5
202314,US,1937,Talton,M,5
202315,US,1937,Theon,M,5
202316,US,1937,Westly,M,5


,state,year,name,M/F,count
0,US,2000,Madison,F,19968.000000
1,US,2000,Olivia,F,12854.000000
2,US,2000,Sydney,F,10244.000000
3,US,2000,Jennifer,F,9389.000000
4,US,2000,Amanda,F,8557.000000
...,...,...,...,...,...
119059,US,2003,Blessen,F,7.766921
119060,US,2003,Ariani,F,10.529099
119061,US,2003,Anadia,F,9.648075
119062,US,2003,Areesha,F,9.530788


,state,year,name,M/F,count
0,US,2000,Madison,F,19968.000000
1,US,2000,Olivia,F,12854.000000
2,US,2000,Sydney,F,10244.000000
3,US,2000,Jennifer,F,9389.000000
4,US,2000,Amanda,F,8557.000000
...,...,...,...,...,...
124533,US,2004,Srija,F,10.092601
124534,US,2004,Anadia,F,8.822809
124535,US,2004,Areesha,F,8.615908
124536,US,2004,Kenniya,F,10.092601


,state,year,name,M/F,count
0,US,2000,Madison,F,19968.000000
1,US,2000,Olivia,F,12854.000000
2,US,2000,Sydney,F,10244.000000
3,US,2000,Jennifer,F,9389.000000
4,US,2000,Amanda,F,8557.000000
...,...,...,...,...,...
130007,US,2005,Xzander,M,4.671317
130008,US,2005,Shuayb,M,4.671317
130009,US,2005,Maddox,M,100.474352
130010,US,2005,Om,M,52.885781


...

,state,year,name,M/F,count
0,US,2000,Madison,F,19968.000000
1,US,2000,Olivia,F,12854.000000
2,US,2000,Sydney,F,10244.000000
3,US,2000,Jennifer,F,9389.000000
4,US,2000,Amanda,F,8557.000000
...,...,...,...,...,...
212117,US,2020,Zennia,F,8.110379
212118,US,2020,Afsheen,F,8.110379
212119,US,2020,Audreyrose,F,8.110379
212120,US,2020,Kennice,F,8.110379


,state,year,name,M/F,count
0,US,2000,Madison,F,19968.000000
1,US,2000,Olivia,F,12854.000000
2,US,2000,Sydney,F,10244.000000
3,US,2000,Jennifer,F,9389.000000
4,US,2000,Amanda,F,8557.000000
...,...,...,...,...,...
217591,US,2021,Marigny,F,8.110379
217592,US,2021,Zenida,F,7.910379
217593,US,2021,Genica,F,7.910379
217594,US,2021,Ni,F,7.910379


,state,median_year,name,M/F,count,cumsum,sum,median_age,thisyear_count,y,year
0,US,1937,Levie,F,6.000000,6.000000,6.000000,65,0.000000,2.000000,2003
1,US,1937,Zenith,M,6.000000,6.000000,6.000000,65,0.000000,2.000000,2003
2,US,1937,Millie,M,5.000000,5.000000,10.000000,65,0.000000,2.000000,2003
3,US,1938,Aurore,F,5.000000,5.000000,10.000000,64,0.000000,2.000000,2003
4,US,1938,Isao,M,7.000000,7.000000,12.000000,64,0.000000,2.000000,2003
...,...,...,...,...,...,...,...,...,...,...,...
109475,US,2018,Eiji,M,4.530965,25.271633,44.179997,3,7.033242,7.910379,2022
109476,US,2018,Selman,M,4.530965,25.271633,44.179997,3,7.033242,7.910379,2022
109477,US,2018,Illia,F,4.530965,25.271633,44.179997,3,7.033242,7.910379,2022
109478,US,2018,Genica,F,6.093365,33.991872,56.036522,3,7.910379,8.620381,2022


,state,median_year,name,M/F,count,cumsum,sum,median_age,thisyear_count,y,year
0,US,1937,Levie,F,6.000000,6.000000,6.000000,65,0.000000,2.0,2003
1,US,1937,Zenith,M,6.000000,6.000000,6.000000,65,0.000000,2.0,2003
2,US,1937,Millie,M,5.000000,5.000000,10.000000,65,0.000000,2.0,2003
3,US,1938,Aurore,F,5.000000,5.000000,10.000000,64,0.000000,2.0,2003
4,US,1938,Isao,M,7.000000,7.000000,12.000000,64,0.000000,2.0,2003
...,...,...,...,...,...,...,...,...,...,...,...
109198,US,2015,Genghis,M,0.774514,11.027500,21.547723,6,2.943371,2.0,2022
109199,US,2015,Salana,F,0.774514,11.067070,21.587293,6,2.943371,2.0,2022
109397,US,2016,Revere,M,1.215863,14.571709,29.093668,5,4.498759,2.0,2022
109398,US,2016,Phoebie,F,1.215863,14.571709,29.093668,5,4.498759,2.0,2022


Predictions for 2003:
Score: 0.07931663199840903
Predictions for 2004:
Score: 0.08999705666392582
Predictions for 2005:
Score: 0.09505296569648283
Predictions for 2006:
Score: 0.10235629774117481
Predictions for 2007:
Score: 0.11029619547276326
Predictions for 2008:
Score: 0.119658627756507
Predictions for 2009:
Score: 0.1278430580360585
Predictions for 2010:
Score: 0.13799396258952426
Predictions for 2011:
Score: 0.14561906555454981
Predictions for 2012:
Score: 0.15301468977711757
Predictions for 2013:
Score: 0.1618797790408436
Predictions for 2014:
Score: 0.1687987216748964
Predictions for 2015:
Score: 0.17680641159934768
Predictions for 2016:
Score: 0.1816771020803558
Predictions for 2017:
Score: 0.1873248159280077
Predictions for 2018:
Score: 0.19271598088627329
Predictions for 2019:
Score: 0.19682486869977378
Predictions for 2020:
Score: 0.1997742463233834
Predictions for 2021:
Score: 0.20541475036618131
Predictions for 2022:
Score: 0.210919169342361
